# complex sine wave
Improvement of [GUI apps in Jupyter using ipywidgets](https://www.youtube.com/watch?v=f0WmLo8AVxo) tutorial

In [1]:
from ipywidgets.widgets import Label, FloatProgress, FloatSlider, Button
from ipywidgets.widgets import Layout, HBox, VBox
from IPython.display import display

import numpy as np
import bqplot as bq
import time
import threading

In [2]:
# flag to control loop
flag_on = True
flag_play = False

#data to plot
x = np.linspace(0, 2*np.pi, 500)
dx = x[1] - x[0]
y = np.sin(x)

## GUI element

In [3]:
# stop button
button_stop = Button(
    description = 'stop',
    icon = 'stop',
    button_style = 'warning',
    layout = Layout(width='100px')
)

def button_stop_click(self):
    global flag_on
    flag_on = False

button_stop.on_click(button_stop_click)
button_stop

Button(button_style='warning', description='stop', icon='stop', layout=Layout(width='100px'), style=ButtonStyl…

In [4]:
# pause/play button
button_play = Button(
    description = 'play',
    icon = 'fa-play',
    button_style = 'primary',
    layout = Layout(width='100px')
)

def button_play_click(self):
    global flag_play
    global button_play
    if button_play.icon == 'pause':
        flag_play = False
        button_play.icon = 'fa-play'
        button_play.description = 'play'
    else:
        flag_play = True
        button_play.icon = 'fa-pause'
        button_play.description = 'pause'

button_play.on_click(button_play_click)
button_play

Button(button_style='primary', description='play', icon='play', layout=Layout(width='100px'), style=ButtonStyl…

In [5]:
# progressbar 
progressbar = FloatProgress(
    value = y[-1],
    min = -1.5,
    max = 1.5,
    description = 'Progress:',
    style = {'desciption_width': 'initial'},
    layout = Layout(width='365px')
)
progressbar

FloatProgress(value=-2.4492935982947064e-16, description='Progress:', layout=Layout(width='365px'), max=1.5, m…

In [6]:
# label
label = Label(
    value = str(np.round(y[-1],2)),
    layout = Layout(width='365px')
)
label

Label(value='-0.0', layout=Layout(width='365px'))

In [7]:
# progressbar_and_label
progressbar_and_label = HBox(
    children = (progressbar, label),
    layout = Layout(margin='0 0 0 43px')
)
progressbar_and_label

In [8]:
# slider
slider = FloatSlider(
    value = 0,
    min = 0,
    max = 0.5,
    step = 0.01,
    description = 'Slider:',
    layout = Layout(width='490px', margin='0 0 5px 0')
)
slider

FloatSlider(value=0.0, description='Slider:', layout=Layout(margin='0 0 5px 0', width='490px'), max=0.5, step=…

In [9]:
# plot

#scale
x_sc = bq.LinearScale()
y_sc = bq.LinearScale()

#axis
x_ax = bq.Axis(
    label = 'x(t)',
    scale = x_sc
)
y_ax = bq.Axis(
    label = 'y(t)',
    scale = y_sc,
    orientation = 'vertical'
)

# line
line = bq.Lines(
    x = x,
    y = y,
    scales = {
        'x': x_sc,'y': y_sc
    }
)

# figure
fig = bq.Figure(
    layout = Layout(width='500px', height='300px'),
    axes = [x_ax, y_ax],
    marks = [line],
    fig_margin = dict(top=10, bottom=40, left=50, right=10)
)

fig

Figure(axes=[Axis(label='x(t)', scale=LinearScale()), Axis(label='y(t)', orientation='vertical', scale=LinearS…

In [10]:
# join
screen_box = VBox(
    children = (fig, progressbar_and_label, slider),
    layout = Layout(border = 'solid 2px gray', width='510px')
)

button_box = HBox(
    children = (button_play,button_stop)
)

app = VBox(
    children = (button_box, screen_box)
)

app

In [11]:
# loop
def work():
    global x
    global y
    
    while flag_on:
        while flag_play:
            # get latest value of slider as noise amplitude
            A = slider.value

            # delete old values from x and y
            x = np.delete(x, 0)
            y = np.delete(y, 0)

            # add new values to x and y
            x = np.append(x, x[-1] + dx)
            noise = A * (np.random.rand() - 0.5)
            y = np.append(y, np.sin(x[-1]) + noise)

            # update progressbar and label
            progressbar.value = y[-1]
            label.value = str(np.round(y[-1],2))

            # update plot
            line.x = x
            line.y = y

            # control cycle speed
            time.sleep(0.05)
            
            if flag_on != True:
                break
    screen_box.close()
    print('App is closed. Please restart the web')

In [12]:
# display the app
display(app)

# set the flag to true
flag = True

# create thread
thread = threading.Thread(target=work)

# start thread
thread.start()